In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
print("PyTorch has version {}".format(torch.__version__))
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

PyTorch has version 2.9.0+cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 67.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import networkx as nx
!rm grafo_prendas.gexf
!wget -O grafo_prendas.gexf https://raw.githubusercontent.com/naaiaafernaandez/Reto_10_Azul_Oscuro/main/grafo_prendas.gexf
graph = nx.read_gexf("grafo_prendas.gexf")

rm: cannot remove 'grafo_prendas.gexf': No such file or directory
--2026-01-19 10:55:16--  https://raw.githubusercontent.com/naaiaafernaandez/Reto_10_Azul_Oscuro/main/grafo_prendas.gexf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83764125 (80M) [text/plain]
Saving to: ‘grafo_prendas.gexf’

grafo_prendas.gexf  100%[===================>]  79.88M   335MB/s    in 0.2s    

2026-01-19 10:55:19 (335 MB/s) - ‘grafo_prendas.gexf’ saved [83764125/83764125]



In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' # Parche para evitar crashes en Windows

import torch
import networkx as nx
import pandas as pd
import numpy as np
import torch_geometric.transforms as T
from torch_geometric.utils import from_networkx, negative_sampling
from torch_geometric.nn import GCNConv
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from torch_geometric.data import Data

# Configuración del dispositivo (CUDA si es posible, si no CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Ejecutando en: {device}")

/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_scatter/_version_cuda.so
  import torch_geometric.typing
/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_sparse/_version_cuda.so
  import torch_geometric.typing


🚀 Ejecutando en: cpu


In [ ]:
list(graph.nodes(data=True))[:2]

[('1',
  {'talla': '3',
   'nivel': 2,
   'sub_category': 'jersey',
   'color_nivel': 54,
   'long_cm': 58.0,
   'sleeve_long_cm': 58.0,
   'tiempo': 'warm',
   'styles': 'boho, classic',
   'prints': 'smooth',
   'fits': 'loose',
   'importance_score': 0.010333766500911052,
   'versatilidad': 79.048,
   'score_hits': 0.00035358236851880235,
   'label': '1'}),
 ('2',
  {'talla': '3',
   'nivel': 2,
   'sub_category': 'jersey',
   'color_nivel': 51,
   'long_cm': 58.0,
   'sleeve_long_cm': 58.0,
   'tiempo': 'warm',
   'styles': 'boho, classic',
   'prints': 'smooth',
   'fits': 'loose',
   'importance_score': 0.00834629337552416,
   'versatilidad': 53.96,
   'score_hits': 0.0002855807237597534,
   'label': '2'})]

In [ ]:
from torch_geometric.transforms import NormalizeFeatures
import random
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx#, from_networkx

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):  
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=100
):
    loss_history = []
    val_auc_history = []
    epochs_history = []
    best_val_auc = 0
    best_epoch = 0
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)
        loss_history.append(loss.item())
        val_auc_history.append(val_auc)
        epochs_history.append(epoch)
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pth')
        if epoch % 10 == 0: 
            print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}")

    print(f"\n¡Entrenamiento finalizado!")
    print(f"Mejor AUC obtenido: {best_val_auc:.4f} en la época {best_epoch}")
    print("El mejor modelo ha sido guardado como 'best_model.pth'")

    # Visualización
    plt.figure(figsize=(10, 6))
    ax1 = plt.gca()
    ax1.plot(epochs_history, loss_history, 'r-', label='Train Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss', color='r')
    ax1.tick_params(axis='y', labelcolor='r')
    ax2 = ax1.twinx()
    ax2.plot(epochs_history, val_auc_history, 'b-', label='Val AUC')
    ax2.set_ylabel('AUC', color='b')
    ax2.tick_params(axis='y', labelcolor='b')
    ax2.plot(best_epoch, best_val_auc, 'go', label='Best Model') # Punto verde
    plt.title('Training History')
    plt.legend(loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.show()
    return model
def eval_link_predictor(model, data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.detach().cpu().numpy())

In [ ]:
import torch
import numpy as np
import pandas as pd 
from torch_geometric.utils import from_networkx
from torch_geometric.transforms import RandomLinkSplit
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

data = from_networkx(graph)
print("--- INICIANDO PROCESAMIENTO DE VARIABLES ---")
nodes_data = list(graph.nodes(data=True))
num_vars = ['nivel','color_nivel']
cat_vars = ['talla','sub_category','tiempo','styles','prints','fits']
print(f"-> Variables Numéricas seleccionadas: {num_vars}")
print(f"-> Variables Categóricas seleccionadas: {cat_vars}")
print("\n--- Procesando Numéricas ---")
num_features = []
for var in num_vars:
    raw_values = [n[1].get(var, 0.0) for n in nodes_data]
    print(f"Variable '{var}' (primeros 5 orig): {raw_values[:5]}")
    col_values = np.array(raw_values).reshape(-1, 1)
    scaler = MinMaxScaler()
    col_values = scaler.fit_transform(col_values)
    num_features.append(col_values)
print("\n--- Procesando Categóricas ---")
cat_features = []
for var in cat_vars:
    raw_values = [str(n[1].get(var, 'unknown')) for n in nodes_data]
    print(f"Variable '{var}' (primeros 5 orig): {raw_values[:5]}")
    encoder = LabelEncoder()
    col_values = encoder.fit_transform(raw_values)
    print(f"   -> Codificado (primeros 5): {col_values[:5]}")
    col_values = col_values.reshape(-1, 1)
    cat_features.append(col_values)
all_features = np.hstack(num_features + cat_features)
data.x = torch.tensor(all_features, dtype=torch.float)
print("\n--- RESULTADO FINAL ---")
print(f"Dimensiones de la matriz de características (Nodos x Variables): {data.x.shape}")
print("Ejemplo de las características del primer nodo (fila 0):")
print(data.x[0])
df_example = pd.DataFrame(data.x.numpy(), columns=num_vars + cat_vars)
print("\nTabla con los primeros 5 nodos procesados:")
print(df_example)
attributes_to_remove = []
for key in data.keys():
    if key not in ['edge_index', 'x', 'y', 'num_nodes', 'edge_label', 'edge_label_index']:
        attributes_to_remove.append(key)
for key in attributes_to_remove:
    if hasattr(data, key):
        delattr(data, key)
split = RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=2.0,)
train_data, val_data, test_data = split(data)
print("\n¡Datos divididos correctamente!")

--- INICIANDO PROCESAMIENTO DE VARIABLES ---
-> Variables Numéricas seleccionadas: ['nivel', 'color_nivel']
-> Variables Categóricas seleccionadas: ['talla', 'sub_category', 'tiempo', 'styles', 'prints', 'fits']

--- Procesando Numéricas ---
Variable 'nivel' (primeros 5 orig): [2, 2, 2, 2, 2]
Variable 'color_nivel' (primeros 5 orig): [54, 51, 51, 100, 100]

--- Procesando Categóricas ---
Variable 'talla' (primeros 5 orig): ['3', '3', '1', '1', '4']
   -> Codificado (primeros 5): [3 3 0 0 4]
Variable 'sub_category' (primeros 5 orig): ['jersey', 'jersey', 'jersey', 'jersey', 'jersey']
   -> Codificado (primeros 5): [5 5 5 5 5]
Variable 'tiempo' (primeros 5 orig): ['warm', 'warm', 'warm', 'warm', 'warm']
   -> Codificado (primeros 5): [2 2 2 2 2]
Variable 'styles' (primeros 5 orig): ['boho, classic', 'boho, classic', 'boho, classic', 'boho, classic', 'boho, classic']
   -> Codificado (primeros 5): [4 4 4 4 4]
Variable 'prints' (primeros 5 orig): ['smooth', 'smooth', 'smooth', 'smooth', 's

In [ ]:
import torch
ruta_guardado = '/content/drive/MyDrive/modelo_gnn_1000.pth'
model = Net(in_channels=data.num_features, hidden_channels=128, out_channels=64)
model.load_state_dict(torch.load(ruta_guardado))
model.eval()

Net(
  (conv1): GCNConv(8, 128)
  (conv2): GCNConv(128, 64)
)

In [ ]:
from funciones import *
recomendador(model, data, graph)

--- INICIANDO MOTOR DE RECOMENDACION ---

--- FICHA TECNICA DEL NUEVO PRODUCTO (Simulado) ---
  COLOR: MUSTAND (ID: 40)
  Fits: loose
  Label: 2574
  Prints: smooth, floral
  Styles: night, classic
  Sub_category: chaqueta
  Talla: 3
  Tiempo: 
  Nivel Asignado: 3 (Por ser chaqueta)

Buscando COINCIDENCIA para CHAQUETA:
   Requisitos: Nivel 3 | Talla 3 (±1)
   Color Objetivo: 40 (Exacto) | Plan B: 40-46
   Encontrado Color Exacto en la posicion #4629!

--- MATCH SELECCIONADO ---
   Simulada -> chaqueta | Talla 3 | Color 40
   Predicha -> chaqueta | Talla 2 | Color mustand (ID 40)
   Tipo de Match: EXACTO

Calculando Combinaciones (Outfits)...

--- MEJORES LOOKS VALIDADOS ---
 Puntuacion     IDs_Ref P1_talla  P1_nivel P1_sub_category  P1_color_nivel P1_tiempo P1_styles    P1_prints  P1_fits P2_talla  P2_nivel P2_sub_category  P2_color_nivel P2_tiempo P2_styles P2_prints  P2_fits
      0.525 2881 + 5710        3         1      pantalones               0      warm    street animal_print s